# Logistic Regression Model
Tristan Miller, 9/18/2019



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Outline-of-model" data-toc-modified-id="Outline-of-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Outline of model</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Features-&amp;-Labels-testing" data-toc-modified-id="Features-&amp;-Labels-testing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Features &amp; Labels testing</a></span></li><li><span><a href="#Scratch" data-toc-modified-id="Scratch-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scratch</a></span></li></ul></div>

## Outline of model

For the minimum viable product, I will use the simplest model: logistic regression.  There will be four classification models:
1. root (12 classes: -1 to 11)
1. character (6 classes: unison, power, maj, min, dim, aug)
1. add (9 classes: min9, 9, 4, tt, min6, 6, min7, maj7, nan)
1. inversion (3 classes: 3, 5, nan)

mp3 data will undergo the following preparation steps:
1. Converted to dB spectra
1. Abridged to be same length as chord file
1. Remove rows with nan root
1. Normalized by song
1. (Training data only) Duplicated 12 times, each with a different transposition 
1. Concatenated, with 'flag' rows between songs
1. After root is predicted, each time step is transposed to have root at same location.  Then other 3 models are applied.

Chord labels will undergo the following preparation steps:
1. 'Blockified' meaning for each interval of fixed length, the most common chord is labeled.
1. Abridged to be same length as mp3 spectra
1. Ignore rows with nan root
1. root, character, add, and inversion extracted from chord string
1. (Training data only) root label transposed as mp3 data is transposed

These steps imply the following model parameters:
1. Time step length
1. range of fequencies in spectra
1. Any classes worth ignoring (e.g. aug and dim)

During validation, I will score the model as follows:

```0.5(F1 of root) +
0.3(accuracy of root)(F1 of character, among data with correct root) +
0.1(accuracy of root & character)(F1 of add) +
0.1(accuracy of root & character)(F1 of inversion)```

The F1 score I intend to use is the weighted F1 (see https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1).

## Setup

In [268]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import sys

In [269]:
if 'chord_loader' in sys.modules:
    del sys.modules['chord_loader']
sys.path.append('.')
import chord_loader

In [3]:
song_directory = pd.read_csv('song_directory.csv')

## Features & Labels testing

In [272]:
features, labels, transposition = chord_loader.get_features_labels(
    song_directory.iloc[0:2,:],0.5,32.7,7,1,True)

In [273]:
features.shape

(7080, 96)

If it takes 6.3 s per song, then this whole thing will take about an hour and a half, and that's before even training the model.

The featurs table will be about (3 million,100), and it will take up about 2 GB.

Using higher `bins_per_note` or shorter `block_length` expands the data size, but appears not to increase the processing time.  Probably most of that processing time is in librosa's load function.

## Scratch

In [255]:
print(chord_loader.get_true_block(0.5,32.7,7,1))

0.5010034403669724


In [256]:
maxfreq = 32.7*(2**7)
sr = 4*maxfreq
hop_restriction = (7-1)**2
hop_length = round(.5*sr/hop_restriction)*hop_restriction
print(hop_length)

8388


In [257]:
print(hop_restriction)

36


In [258]:
print((7-1)**2)

36
